In [5]:
import numpy as np
import MITgcmutils

path_grid = "ECCO2/LLC270/ECCO-Darwin_extension/grid"
path_3d = "ECCO2/LLC270/ECCO-Darwin_extension/budget/average_DIC_3d"
path_2d = "ECCO2/LLC270/ECCO-Darwin_extension/budget/average_2d"

#### Variable list

In [8]:
fldList_3d = ['TRAC01  ','ADVxTr01','ADVyTr01','ADVrTr01','DFxETr01','DFyETr01','DFrETr01',
              'DFrITr01','gDAR01  ','cDIC    ','cDIC_PIC','respDIC ','rDIC_DOC','rDIC_POC','dDIC_PIC']
fldList_2d = ['ETAN    ','oceFWflx','SFLUX   ','TFLUX   ','oceSPflx','oceQsw  ','fluxCO2 ','gDICsurf',
           'gDICEpr ','gALKEpr ','gNO3Epr ','gNO2Epr ','gNH4Epr ','gO2Epr  ','gPO4Epr ','gFeTEpr ','gSiO2Epr']

mapping_3d = zip(range(len(fldList_3d)),fldList_3d)
mapping_2d = zip(range(len(fldList_2d)),fldList_2d)

display(list(mapping_3d))
print()
display(list(mapping_2d))

[(0, 'TRAC01  '),
 (1, 'ADVxTr01'),
 (2, 'ADVyTr01'),
 (3, 'ADVrTr01'),
 (4, 'DFxETr01'),
 (5, 'DFyETr01'),
 (6, 'DFrETr01'),
 (7, 'DFrITr01'),
 (8, 'gDAR01  '),
 (9, 'cDIC    '),
 (10, 'cDIC_PIC'),
 (11, 'respDIC '),
 (12, 'rDIC_DOC'),
 (13, 'rDIC_POC'),
 (14, 'dDIC_PIC')]

[(0, 'ETAN    '),
 (1, 'oceFWflx'),
 (2, 'SFLUX   '),
 (3, 'TFLUX   '),
 (4, 'oceSPflx'),
 (5, 'oceQsw  '),
 (6, 'fluxCO2 '),
 (7, 'gDICsurf'),
 (8, 'gDICEpr '),
 (9, 'gALKEpr '),
 (10, 'gNO3Epr '),
 (11, 'gNO2Epr '),
 (12, 'gNH4Epr '),
 (13, 'gO2Epr  '),
 (14, 'gPO4Epr '),
 (15, 'gFeTEpr '),
 (16, 'gSiO2Epr')]

#### importing data

In [12]:
d_2 = MITgcmutils.mds.rdmds(f"{path_2d}/average_2d.0000002232",rec=[6,8])
d_3 = MITgcmutils.mds.rdmds(f"{path_3d}/average_DIC_3d.0000002232",rec=[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14])
depth_c = MITgcmutils.mds.rdmds(f"{path_grid}/RC")
depth_f = MITgcmutils.mds.rdmds(f"{path_grid}/RF")

#### Flatten 3d data

In [76]:
# Calculating delta depth
delta_depth_c = np.diff(depth_c, axis=0) * -1
delta_depth_c = delta_depth_c[:,0,0]

delta_depth_f = np.diff(depth_f, axis=0) * -1
delta_depth_f = delta_depth_f[:,0,0]

# getting index of top 100m depth
idx_top_100m = np.where(depth_c>=-100)[0]

In [138]:
# Using numpy
delta_depth_f_vec = delta_depth_f[idx_top_100m, np.newaxis, np.newaxis]
flattened_d3 = np.sum(delta_depth_f_vec * d_3[:,idx_top_100m,:,:], axis=1)/100 # Compressing the depth into 2D for top 100m

In [146]:
# Using loop
flattened_d3_test = np.zeros([13, 3510, 270])

for variable in range(d_3.shape[0]):
    for idx,delta_depth in enumerate(delta_depth_f[idx_top_100m]):
        flattened_d3_test[variable,:,:] += d_3[variable,idx,:,:] * delta_depth

flattened_d3_test = flattened_d3_test/100
print(flattened_d3_test.shape)

(13, 3510, 270)


#### Combine variables

In [152]:
combined_data = np.concatenate((flattened_d3, d_2), axis=0)
combined_data.shape

(15, 3510, 270)

#### plotting

#### Excel Test

In [58]:
delta_depth_c = d_3[1]
variable_1_flattened = variable_1_data.reshape(variable_1_data.shape[0], -1)  # Shape: (50, 949770)

# Combine delta depth values with the flattened data
combined_data = np.column_stack((delta_depth_f, variable_1_flattened))

# Convert to DataFrame for better Excel formatting
column_names = ['Delta Depth'] + [f'Value_{i}' for i in range(variable_1_flattened.shape[1])]
df_excel = pd.DataFrame(combined_data, columns=column_names).T

# Save to Excel
excel_path = "variable_1_depth_comparison.xlsx"
df_excel.to_excel(excel_path, index=False)
